### Intial setup

Installation of libraries

In [ ]:
!pip install -U autogen-agentchat autogen-ext[openai]

API key setup

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "your-openai-api-key"

### Multi Agent Setup

In [ ]:
from autogen.agentchat import AssistantAgent

Fine tuned Agents

Semantic Agent

In [ ]:
# !pip install torch transformers sentencepiece

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [ ]:
#Takes 30s to load the model 
from agents.semantic_agent.semantic_agent import SemanticAgent

semantic_agent = SemanticAgent(
    model_dir="agents/semantic_agent/t5_semantic_agent_final"
)

/Users/trushatalati/micromamba/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from agents.contrast_agent.contrast_agent import ContrastAgent

contrast_agent = ContrastAgent(
    model_dir="virajns2/contrast-violation-t5"
)

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the fine-tuned GPT-2 model and tokenizer
model = GPT2LMHeadModel.from_pretrained("path_to_model") #TODO give path to model
tokenizer = GPT2Tokenizer.from_pretrained("path_to_your_fine_tuned_model")#TODO give path to tokenizer

In [ ]:
from autogen.agentchat import AssistantAgent

class CustomGPT2Agent(AssistantAgent):
    def __init__(self, name, model, tokenizer):
        super().__init__(name=name)
        self.model = model
        self.tokenizer = tokenizer

    def generate_reply(self, messages):
        # Extract the latest user message
        user_message = messages[-1]['content']
        
        # Tokenize the input
        inputs = self.tokenizer.encode(user_message, return_tensors='pt')
        
        # Generate a response
        outputs = self.model.generate(inputs, max_length=150, num_return_sequences=1)
        
        # Decode the generated response
        reply = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        return reply

In [ ]:
custom_agent = CustomGPT2Agent(name="CustomGPT2Agent", model=model, tokenizer=tokenizer)

GPT based Agents

In [ ]:
# Visually Impaired Agent
visually_impaired_agent = AssistantAgent(
    name="VisuallyImpairedAgent",
    system_message="Simulate a user navigating via screen reader. Identify missing labels, improper heading structures, and ARIA attribute issues.",
    llm_config={"model": "gpt-4", "temperature": 0}
)

In [ ]:
# Motor-Impaired Agent
motor_impaired_agent = AssistantAgent(
    name="MotorImpairedAgent",
    system_message="Simulate keyboard-only navigation. Detect elements not accessible via keyboard and lack of focus indicators.",
    llm_config={"model": "gpt-4", "temperature": 0}
)


In [ ]:
# Color-Blind Agent
color_blind_agent = AssistantAgent(
    name="ColorBlindAgent",
    system_message="Analyze color usage for accessibility. Identify issues with color contrast and reliance on color cues.",
    llm_config={"model": "gpt-4", "temperature": 0}
)

In [ ]:
fixing_agent = AssistantAgent(
    name="FixingAgent",
    system_message="Provide code-level fixes for identified accessibility issues based on best practices and guidelines, based on the problems stated by everyone",
    llm_config={"model": "gpt-4", "temperature": 0}
)

Configure Agent Interactions

In [ ]:
from autogen.agentchat import UserProxyAgent, GroupChat, GroupChatManager

user_proxy = UserProxyAgent(
    name="UserProxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=3,
    code_execution_config={"use_docker": False}
)

group_chat = GroupChat(
    agents=[user_proxy, semantic_agent, contrast_agent, visually_impaired_agent, motor_impaired_agent, color_blind_agent, fixing_agent],
    messages=[],
    max_round=5
)

manager = GroupChatManager(groupchat=group_chat, llm_config={"model": "gpt-4"})

Run the Evaluation

In [ ]:
ui = "" #TODO: FETCH RESPECTIVE DATA FROM TEST SET AND PASS TO SYSTEM

user_proxy.initiate_chat(manager, message = ui)

Feed back loop

In [ ]:
for msg in group_chat.messages:
    if msg['role'] != 'FixingAgent':
        fixing_agent.receive_message(msg)

for msg in group_chat.messages:
    print(f"{msg['role']}: {msg['content']}\n")